<a href="https://colab.research.google.com/github/AdiVM/Neuro240/blob/main/Neuro240AlmostThereCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# All future runs can start here
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import os

Mounted at /content/drive


In [ ]:
metadata_path = "/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/Data_Entry_2017_v2020.csv"
image_folder = "/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/images"

metadata = pd.read_csv(metadata_path)

print("Metdata loaded")

# Filtering the metadata to find images labeled either no finding or those containing the word mass
filtered_metadata = metadata[
    (metadata["Finding Labels"] == "No Finding") |
    (metadata["Finding Labels"].str.contains("Mass", na=False))
]

filtered_image_indexes = set(filtered_metadata["Image Index"])
filtered_metadata = filtered_metadata.head(50000)

matching_images = sorted(list(filtered_metadata["Image Index"]))

# Convert to stored list
matching_images = sorted(list(matching_images))

print(f"Total matching images found: {len(matching_images)}")

Metdata loaded
Total matching images found: 50000


In [ ]:
# Now to perform stratified shuffle split
from sklearn.model_selection import train_test_split
import pandas as pd

 # Check class distribution before splitting
print(filtered_metadata["Finding Labels"].value_counts())

# There are many small classes of mass, so need to group them all together before splitting
# Standardize labels: Convert anything containing "Mass" to just "Mass"
filtered_metadata["Finding Labels"] = filtered_metadata["Finding Labels"].apply(
    lambda x: "Mass" if "Mass" in x else x
)

# Verify new label counts
print(filtered_metadata["Finding Labels"].value_counts())


Finding Labels
No Finding                                                                          45542
Mass                                                                                 1708
Infiltration|Mass                                                                     329
Mass|Nodule                                                                           293
Effusion|Mass                                                                         285
                                                                                    ...  
Effusion|Emphysema|Mass|Nodule                                                          1
Atelectasis|Consolidation|Effusion|Fibrosis|Infiltration|Mass|Pleural_Thickening        1
Atelectasis|Consolidation|Effusion|Fibrosis|Infiltration|Mass                           1
Cardiomegaly|Consolidation|Effusion|Infiltration|Mass|Nodule                            1
Edema|Fibrosis|Infiltration|Mass                                                     

In [ ]:
label_map = {"No Finding": 0, "Mass": 1}
filtered_metadata["Label"] = filtered_metadata["Finding Labels"].map(label_map)

In [ ]:
# Split the data while ensuring proportional distribution of classes
train_metadata, test_metadata = train_test_split(
    filtered_metadata,
    test_size=0.2,
    stratify=filtered_metadata["Label"],
    random_state=42
)

# Class distribution in train and test sets
print("Training set:")
print(train_metadata["Label"].value_counts())

print("Testing Set:")
print(test_metadata["Label"].value_counts())

Training set:
Label
0    36434
1     3566
Name: count, dtype: int64
Testing Set:
Label
0    9108
1     892
Name: count, dtype: int64


In [ ]:
print(f"Train metadata entries: {len(train_metadata)}")
print(f"Test metadata entries: {len(test_metadata)}")

Train metadata entries: 40000
Test metadata entries: 10000


In [ ]:
# Filtering metdata
# Convert "Image Index" column to a set for fast lookup
train_image_files = set(train_metadata["Image Index"])
test_image_files = set(test_metadata["Image Index"])

train_images = sorted(list(train_image_files))
test_images = sorted(list(test_image_files))

# Convert to sorted lists for consistency
train_images = sorted(list(train_images))
test_images = sorted(list(test_images))


print(f"Total train images found: {len(train_images)}")
print(f"Total test images found: {len(test_images)}")

# Print a few samples
print("Sample train images:", train_images[:10])
print("Sample test images:", test_images[:10])

Total train images found: 40000
Total test images found: 10000
Sample train images: ['00000002_000.png', '00000004_000.png', '00000005_000.png', '00000005_002.png', '00000005_003.png', '00000005_004.png', '00000005_005.png', '00000006_000.png', '00000007_000.png', '00000011_002.png']
Sample test images: ['00000005_001.png', '00000008_001.png', '00000011_001.png', '00000011_003.png', '00000013_000.png', '00000013_017.png', '00000013_029.png', '00000013_030.png', '00000018_000.png', '00000032_049.png']


In [ ]:
# Will use TensorFlow for model training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

In [ ]:
# Image preprocessing parameters
image_size = (224, 224)  # Resize images
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Only rescale for testing
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load train images from directory
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_metadata,
    directory=image_folder,
    x_col="Image Index",
    y_col="Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="raw"
)

# Load test images
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_metadata,
    directory=image_folder,
    x_col="Image Index",
    y_col="Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="raw"
)

Found 39996 validated image filenames.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 4 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  warnings.warn(


Found 10000 validated image filenames.


In [ ]:
print("Train distribution:")
print(train_metadata["Label"].value_counts())

print("\nTest distribution:")
print(test_metadata["Label"].value_counts())

Train distribution:
Label
0    36434
1     3566
Name: count, dtype: int64

Test distribution:
Label
0    9108
1     892
Name: count, dtype: int64


In [ ]:
# def train_model_with_subset(train_metadata, test_metadata, image_folder,
#                             train_size,
#                             image_size=(224, 224), batch_size=32, epochs=5,
#                             output_dir="/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/results"):
#     import os
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import seaborn as sns
#     from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
#     from tensorflow.keras.preprocessing.image import ImageDataGenerator

#     # Sample training and test subsets
#     train_metadata_subset = train_metadata.sample(train_size, random_state=42)
#     test_metadata_subset = test_metadata.sample(int(train_size / 4), random_state=42)

#     os.makedirs(output_dir, exist_ok=True)
#     result_prefix = os.path.join(output_dir, f"train_{train_size}")

#     # Save class distributions
#     def save_distribution(df, name):
#         counts = df["Finding Labels"].value_counts()
#         percents = counts / counts.sum() * 100
#         dist_df = pd.DataFrame({"Count": counts, "Percent": percents})
#         dist_df.to_csv(f"{result_prefix}_{name}_distribution.csv")
#         return dist_df

#     train_dist = save_distribution(train_metadata_subset, "train")
#     test_dist = save_distribution(test_metadata_subset, "test")

#     # Combined barplot
#     combined_df = pd.concat([
#         train_dist["Percent"].rename("Train"),
#         test_dist["Percent"].rename("Test")
#     ], axis=1).fillna(0).reset_index()

#     # Rename the correct column to 'Label' (usually the first column)
#     combined_df.columns.values[0] = "Label"

#     combined_df = pd.melt(combined_df, id_vars="Label", var_name="Set", value_name="Percent")

#     plt.figure(figsize=(6, 4))
#     sns.barplot(data=combined_df, x="Label", y="Percent", hue="Set")
#     plt.title("Class Distribution: Train vs Test")
#     plt.ylabel("Percentage")
#     plt.xlabel("Class Label")
#     plt.tight_layout()
#     plt.savefig(f"{result_prefix}_class_distribution_comparison.png")
#     plt.close()

#     # Image data generators
#     train_datagen = ImageDataGenerator(
#         rescale=1.0 / 255,
#         rotation_range=15,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         horizontal_flip=True
#     )
#     test_datagen = ImageDataGenerator(rescale=1.0 / 255)

#     train_generator = train_datagen.flow_from_dataframe(
#         dataframe=train_metadata_subset,
#         directory=image_folder,
#         x_col="Image Index",
#         y_col="Finding Labels",
#         target_size=image_size,
#         batch_size=batch_size,
#         class_mode="binary",
#         shuffle=True
#     )
#     test_generator = test_datagen.flow_from_dataframe(
#         dataframe=test_metadata_subset,
#         directory=image_folder,
#         x_col="Image Index",
#         y_col="Finding Labels",
#         target_size=image_size,
#         batch_size=batch_size,
#         class_mode="binary",
#         shuffle=False
#     )

#     # CNN model
#     model = Sequential([
#         Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
#         MaxPooling2D(pool_size=(2, 2)),
#         Conv2D(64, (3, 3), activation="relu"),
#         MaxPooling2D(pool_size=(2, 2)),
#         Conv2D(128, (3, 3), activation="relu"),
#         MaxPooling2D(pool_size=(2, 2)),
#         Flatten(),
#         Dense(128, activation="relu"),
#         Dropout(0.5),
#         Dense(1, activation="sigmoid")
#     ])

#     model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
#     history = model.fit(
#         train_generator,
#         validation_data=test_generator,
#         epochs=epochs,
#         verbose=1
#     )

#     test_loss, test_acc = model.evaluate(test_generator)
#     y_pred_proba = model.predict(test_generator).flatten()
#     y_pred = (y_pred_proba > 0.5).astype(int)
#     y_true = test_generator.classes
#     auc = roc_auc_score(y_true, y_pred_proba)
#     cm = confusion_matrix(y_true, y_pred)

#     # Save confusion matrix
#     confusion_df = pd.DataFrame(
#     cm,
#     index=["Mass", "No Finding"],
#     columns=["Predicted Mass", "Predicted No Finding"])

#     confusion_df.to_csv(f"{result_prefix}_confusion_matrix.csv")

#     # Save predictions
#     filenames = test_generator.filenames
#     results_df = pd.DataFrame({
#         "Filename": filenames,
#         "TrueLabel": y_true,
#         "PredictedLabel": y_pred,
#         "PredictedProb": y_pred_proba
#     })
#     results_df.to_csv(f"{result_prefix}_predictions.csv", index=False)

#     # Save ROC curve
#     fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
#     plt.figure()
#     plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
#     plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title("ROC Curve")
#     plt.legend()
#     plt.grid(True)
#     plt.savefig(f"{result_prefix}_roc_curve.png")
#     plt.close()

#     print(f"Test Accuracy (train size={train_size}): {test_acc * 100:.2f}%")
#     print(f"AUC: {auc:.4f}")

#     return model, history, test_acc, auc

In [ ]:
# model, history, acc, auc = train_model_with_subset(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=500
# )

In [ ]:
# model_250, history_250, acc_250, auc_250 = train_model_with_subset(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=250
# )

In [ ]:
# model_1000, history_1000, acc_1000, auc_1000 = train_model_with_subset(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=1000
# )

In [ ]:
# model_250, history_250, acc_250, auc_250 = train_model_with_noise(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=500
# )

In [ ]:
# def train_model_with_class_weighting(train_metadata, test_metadata, image_folder,
#                             train_size,
#                             image_size=(224, 224), batch_size=32, epochs=5,
#                             output_dir="/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/results_class_weight"):
#     import os
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import seaborn as sns
#     import numpy as np
#     from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
#     from tensorflow.keras.preprocessing.image import ImageDataGenerator

#     # Sample training and test subsets
#     half_size = train_size // 2
#     mass_subset = train_metadata[train_metadata["Finding Labels"] == "Mass"].sample(half_size, random_state=42)
#     no_finding_subset = train_metadata[train_metadata["Finding Labels"] == "No Finding"].sample(half_size, random_state=42)
#     train_metadata_subset = pd.concat([mass_subset, no_finding_subset]).sample(frac=1, random_state=42)  # shuffle

#     test_metadata_subset = test_metadata.sample(int(train_size / 4), random_state=42)

#     os.makedirs(output_dir, exist_ok=True)
#     result_prefix = os.path.join(output_dir, f"train_{train_size}")

#     # Save class distributions
#     def save_distribution(df, name):
#         counts = df["Finding Labels"].value_counts()
#         percents = counts / counts.sum() * 100
#         dist_df = pd.DataFrame({"Count": counts, "Percent": percents})
#         dist_df.to_csv(f"{result_prefix}_{name}_distribution.csv")
#         return dist_df

#     train_dist = save_distribution(train_metadata_subset, "train")
#     test_dist = save_distribution(test_metadata_subset, "test")

#     # Combined barplot
#     combined_df = pd.concat([
#         train_dist["Percent"].rename("Train"),
#         test_dist["Percent"].rename("Test")
#     ], axis=1).fillna(0).reset_index()

#     # Rename the correct column to 'Label' (usually the first column)
#     combined_df.columns.values[0] = "Label"

#     combined_df = pd.melt(combined_df, id_vars="Label", var_name="Set", value_name="Percent")

#     plt.figure(figsize=(6, 4))
#     sns.barplot(data=combined_df, x="Label", y="Percent", hue="Set")
#     plt.title("Class Distribution: Train vs Test")
#     plt.ylabel("Percentage")
#     plt.xlabel("Class Label")
#     plt.tight_layout()
#     plt.savefig(f"{result_prefix}_class_distribution_comparison.png")
#     plt.close()

#     # Image data generators
#     train_datagen = ImageDataGenerator(
#         rescale=1.0 / 255,
#         rotation_range=15,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         horizontal_flip=False
#     )
#     test_datagen = ImageDataGenerator(rescale=1.0 / 255)

#     train_generator = train_datagen.flow_from_dataframe(
#         dataframe=train_metadata_subset,
#         directory=image_folder,
#         x_col="Image Index",
#         y_col="Finding Labels",
#         target_size=image_size,
#         batch_size=batch_size,
#         class_mode="binary",
#         shuffle=False
#     )
#     test_generator = test_datagen.flow_from_dataframe(
#         dataframe=test_metadata_subset,
#         directory=image_folder,
#         x_col="Image Index",
#         y_col="Finding Labels",
#         target_size=image_size,
#         batch_size=batch_size,
#         class_mode="binary",
#         shuffle=False
#     )

#     # CNN model
#     model = Sequential([
#         Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
#         MaxPooling2D(pool_size=(2, 2)),
#         Conv2D(64, (3, 3), activation="relu"),
#         MaxPooling2D(pool_size=(2, 2)),
#         Conv2D(128, (3, 3), activation="relu"),
#         MaxPooling2D(pool_size=(2, 2)),
#         Flatten(),
#         Dense(128, activation="relu"),
#         Dropout(0.5),
#         Dense(1, activation="sigmoid")
#     ])

#     model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

#     history = model.fit(
#     train_generator,
#     validation_data=test_generator,
#     epochs=epochs,
#     verbose=1)

#     test_loss, test_acc = model.evaluate(test_generator)
#     y_pred_proba = model.predict(test_generator).flatten()
#     from sklearn.metrics import f1_score

#     # Trying multiple thresholds to find the optimal one
#     y_true = test_generator.classes

#     thresholds = np.linspace(0.1, 0.9, 9)
#     best_f1 = 0
#     best_threshold = 0.5  # default
#     best_preds = None

#     for t in thresholds:
#       preds = (y_pred_proba > t).astype(int)
#       f1 = f1_score(y_true, preds)
#       print(f"Threshold {t:.2f} → F1 Score: {f1:.4f}")
#       if f1 > best_f1:
#           best_f1 = f1
#           best_threshold = t
#           best_preds = preds

#     print(f"\nBest Threshold: {best_threshold:.2f} → F1 Score: {best_f1:.4f}")


#     auc = roc_auc_score(y_true, y_pred_proba)
#     cm = confusion_matrix(y_true, best_preds)

#     # Save confusion matrix
#     confusion_df = pd.DataFrame(
#     cm,
#     index=["Mass", "No Finding"],
#     columns=["Predicted Mass", "Predicted No Finding"])

#     confusion_df.to_csv(f"{result_prefix}_confusion_matrix.csv")

#     # Save predictions
#     filenames = test_generator.filenames
#     results_df = pd.DataFrame({
#         "Filename": filenames,
#         "TrueLabel": y_true,
#         "PredictedLabel": best_preds,
#         "PredictedProb": y_pred_proba
#     })
#     results_df.to_csv(f"{result_prefix}_predictions.csv", index=False)

#     # Save ROC curve
#     fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
#     plt.figure()
#     plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
#     plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title("ROC Curve")
#     plt.legend()
#     plt.grid(True)
#     plt.savefig(f"{result_prefix}_roc_curve.png")
#     plt.close()

#     print(f"Test Accuracy (train size={train_size}): {test_acc * 100:.2f}%")
#     print(f"AUC: {auc:.4f}")

#     with open(f"{result_prefix}_best_threshold.txt", "w") as f:
#       f.write(f"Best threshold: {best_threshold:.2f}, F1: {best_f1:.4f}")

#     return model, history, test_acc, auc

In [ ]:
# model_250_class, history_250_class, acc_250_class, auc_250_class = train_model_with_class_weighting(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=250
# )

In [ ]:
# model_500_class, history_500_class, acc_500_class, auc_500_class = train_model_with_class_weighting(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=500
# )

In [ ]:
# model_500_class, history_500_class, acc_500_class, auc_500_class = train_model_with_class_weighting(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=7000
# )

In [ ]:
# model_1000_class, history_1000_class, acc_1000_class, auc_1000_class = train_model_with_class_weighting(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=1000
# )

In [ ]:
# model_500_class, history_500_class, acc_500_class, auc_500_class = train_model_with_class_weighting(
#     train_metadata=train_metadata,
#     test_metadata=test_metadata,
#     image_folder=image_folder,
#     train_size=500
# )

In [ ]:
def train_model_with_noise(train_metadata, test_metadata, image_folder,
                            train_size,
                            image_size=(224, 224), batch_size=32, epochs=5,
                            output_dir="/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/results_noise"):
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # Sample training and test subsets
    train_metadata_subset = train_metadata.sample(train_size, random_state=42)
    test_metadata_subset = test_metadata.sample(int(train_size / 4), random_state=42)

    os.makedirs(output_dir, exist_ok=True)
    result_prefix = os.path.join(output_dir, f"train_{train_size}")

    # Save class distributions
    def save_distribution(df, name):
        counts = df["Finding Labels"].value_counts()
        percents = counts / counts.sum() * 100
        dist_df = pd.DataFrame({"Count": counts, "Percent": percents})
        dist_df.to_csv(f"{result_prefix}_{name}_distribution.csv")
        return dist_df

    train_dist = save_distribution(train_metadata_subset, "train")
    test_dist = save_distribution(test_metadata_subset, "test")

    # Combined barplot
    combined_df = pd.concat([
        train_dist["Percent"].rename("Train"),
        test_dist["Percent"].rename("Test")
    ], axis=1).fillna(0).reset_index()

    # Rename the correct column to 'Label' (usually the first column)
    combined_df.columns.values[0] = "Label"

    combined_df = pd.melt(combined_df, id_vars="Label", var_name="Set", value_name="Percent")

    plt.figure(figsize=(6, 4))
    sns.barplot(data=combined_df, x="Label", y="Percent", hue="Set")
    plt.title("Class Distribution: Train vs Test")
    plt.ylabel("Percentage")
    plt.xlabel("Class Label")
    plt.tight_layout()
    plt.savefig(f"{result_prefix}_class_distribution_comparison.png")
    plt.close()


    # Data generators with noisey training set
    train_datagen_noisy = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=20,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        zoom_range=0.1,
        preprocessing_function=lambda x: x + np.random.normal(loc=0.0, scale=0.05, size=x.shape)
    )
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_metadata_subset,
        directory=image_folder,
        x_col="Image Index",
        y_col="Finding Labels",
        target_size=image_size,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True
    )
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_metadata_subset,
        directory=image_folder,
        x_col="Image Index",
        y_col="Finding Labels",
        target_size=image_size,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=False
    )

    # CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    from sklearn.utils.class_weight import compute_class_weight
    import numpy as np

    # Map labels to integers using the generator's class indices
    label_map = train_generator.class_indices
    inv_label_map = {v: k for k, v in label_map.items()}

    # Convert y labels to 0/1
    y_train_labels = train_metadata_subset["Finding Labels"].map(label_map)

    # Compute class weights
    # class_weights = compute_class_weight(
    #     class_weight="balanced",
    #     classes=np.unique(y_train_labels),
    #     y=y_train_labels
    # )
    # class_weight_dict = dict(enumerate(class_weights))

    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=epochs,
        verbose=1
        # class_weight=class_weight_dict
    )

    test_loss, test_acc = model.evaluate(test_generator)
    y_pred_proba = model.predict(test_generator).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)
    y_true = test_generator.classes
    auc = roc_auc_score(y_true, y_pred_proba)
    cm = confusion_matrix(y_true, y_pred)

    # Save confusion matrix
    confusion_df = pd.DataFrame(
    cm,
    index=["Mass", "No Finding"],
    columns=["Predicted Mass", "Predicted No Finding"])

    confusion_df.to_csv(f"{result_prefix}_confusion_matrix.csv")

    # Save predictions
    filenames = test_generator.filenames
    results_df = pd.DataFrame({
        "Filename": filenames,
        "TrueLabel": y_true,
        "PredictedLabel": y_pred,
        "PredictedProb": y_pred_proba
    })
    results_df.to_csv(f"{result_prefix}_predictions.csv", index=False)

    # Save ROC curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{result_prefix}_roc_curve.png")
    plt.close()

    print(f"Test Accuracy (train size={train_size}): {test_acc * 100:.2f}%")
    print(f"AUC: {auc:.4f}")

    return model, history, test_acc, auc

In [ ]:
model_250, history_250, acc_250, auc_250 = train_model_with_noise(
    train_metadata=train_metadata,
    test_metadata=test_metadata,
    image_folder=image_folder,
    train_size=250
)

Found 250 validated image filenames belonging to 2 classes.
Found 62 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 224s 27s/step - accuracy: 0.8266 - loss: 0.4115 - val_accuracy: 0.8548 - val_loss: 0.5413
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9211 - loss: 0.2975 - val_accuracy: 0.8548 - val_loss: 0.5429
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 994ms/step - accuracy: 0.9231 - loss: 0.3176 - val_accuracy: 0.8548 - val_loss: 0.4259
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9313 - loss: 0.2669 - val_accuracy: 0.8548 - val_loss: 0.5195
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9238 - loss: 0.2791 - val_accuracy: 0.8548 - val_loss: 0.5172
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step - accuracy: 0.8303 - loss: 0.5967
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 521ms/step
Test Accuracy (train size=250): 85.48%
AUC: 0.4235


In [ ]:
model_250, history_250, acc_250, auc_250 = train_model_with_noise(
    train_metadata=train_metadata,
    test_metadata=test_metadata,
    image_folder=image_folder,
    train_size=500
)

Found 500 validated image filenames belonging to 2 classes.
Found 125 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 269s 17s/step - accuracy: 0.8092 - loss: 0.5473 - val_accuracy: 0.8880 - val_loss: 0.3567
Epoch 2/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9117 - loss: 0.3336 - val_accuracy: 0.8880 - val_loss: 0.4541
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9146 - loss: 0.3290 - val_accuracy: 0.8880 - val_loss: 0.3977
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9110 - loss: 0.3072 - val_accuracy: 0.8880 - val_loss: 0.3587
Epoch 5/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9278 - loss: 0.2951 - val_accuracy: 0.8880 - val_loss: 0.3953
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 547ms/step - accuracy: 0.8583 - loss: 0.4476
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 563ms/step
Test Accuracy (train size=500): 88.80%
AUC: 0.4344


In [ ]:
model_250, history_250, acc_250, auc_250 = train_model_with_noise(
    train_metadata=train_metadata,
    test_metadata=test_metadata,
    image_folder=image_folder,
    train_size=1000
)